In [1]:
import requests
import re
import datetime
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
def listToInt(daList):
    lisk = daList[:]
    for i in range(len(lisk)):
        lisk[i] = int(lisk[i])
    return lisk

In [3]:
def tld(daList):
    '''typeListDistiller'''
    lisk = daList[:]
    outTypes = []
    outSubtypes = []
    
    #First we'll append to types, then once we see that dash thing, we append to subtypes
    preDash = True
    for i in range(len(lisk)):
        #print(lisk[i])
        if lisk[i] == "" or lisk[i] == " ":
            pass
        elif lisk[i] == "—":
            preDash = False
        else:
            if preDash == True:
                outTypes.append(lisk[i])
            else:
                outSubtypes.append(lisk[i])
    return outTypes, outSubtypes

In [4]:
def removePunctuationFromList(listPlz):
    '''this function removes characters not supported by mtggoldfish.com.  As a note, they do support dashes.
    This function may not be 100% accurate as of now due to lack of testing symbols.'''
    lisk = listPlz[:]
    punctuation = [".",",","'",'"',"?","!","/","%","$","#","@","^","&","(",")","_","+","*"]
    for i in range(len(lisk)):
        for p in punctuation:
            lisk[i] = lisk[i].replace(p,"")
    return lisk

In [5]:
def spacesToPlus(listPlz):
    '''The URLS on mtggoldfish.com require plus-separated names for cards instead of space-separated ones.
    Also, this function handles the weird cards that appear as Word1 // Word2 (Word#) on gatherer.wizards.com'''
    lisk = listPlz[:]
    #replacing regular spaces with +
    for i in range(len(lisk)):
        lisk[i] = lisk[i].replace(" ","+")
        
    # handling double plus exception    
    for i in range(len(lisk)):
        if "++" in lisk[i]:
            lisk[i] = lisk[i].split("+")[0]+"+"+lisk[i].split("+")[2]
    
    # handling other part of double plus exception where the card name occurs twice.
    removeList = []
    for i in range(len(lisk)):
        if lisk.count(lisk[i]) == 2 and lisk[i] not in removeList:
            removeList.append(lisk[i])
    for i in removeList:
        lisk.remove(i)
    return lisk

In [58]:
def plusToSpaces(listPlz):
    for i in range(len(listPlz)):
        listPlz[i] = listPlz[i].replace("+"," ")
    return listPlz

In [45]:
def strippingImages(daStr):
    #print("daStr",daStr)
    #toDelete = re.findall("\<i.*\/\>",daStr)
    toDelete = re.findall('<img src=".[^\<]*>',daStr)
    toReplace = re.findall('alt="[\w ]+"',daStr)
    for i in range(len(toReplace)):
        toReplace[i] = toReplace[i].split('"')[1]
    #print(toReplace)
    #print("toDelete pre export",toDelete)
    for i in range(len(toDelete)):
        daStr = daStr.replace(toDelete[i]," " + toReplace[i].lower() + "Mana ")
    return daStr

In [46]:
def fixingNumbersSoTheyStay(daStr):
    numWordsTo20 = ["zero","one","two","three","four","five",
                    "six","seven","eight","nine","ten",
                  "eleven","twelve","thirteen","fourteen","fifteen",
                  "sixteen","seventeen","eighteen","nineteen","twenty"]
    numNumbersTo20 = list(range(21))
    for i in range(len(numNumbersTo20)):
        daStr = daStr.replace(str(numNumbersTo20[i])," "+numWordsTo20[i]+" ")
    daStr = daStr.replace("+"," plus ")
    daStr = daStr.replace("-"," minus ")
    daStr = daStr.replace("/"," slashDivider ")
    return daStr

In [47]:
def importantRulesTextExtraction(daStr):
    daStr = daStr.replace('<p>'," ")
    daStr = daStr.replace('</p>'," ")
    daStr = daStr.replace('<i>('," ")
    daStr = daStr.replace(')</i>'," ")
    daStr = strippingImages(daStr)
    daStr = fixingNumbersSoTheyStay(daStr)
    return daStr

In [48]:
def getRareAndMythicInfo(setName):
    '''this function takes one parameter, the name of the MtG set.
    After you input the name of the set, it will grab all the names of the rares and mythics from
    gatherer.wizards.com and output them into 2 lists, rareNames and mythicNames.'''
    
    #If their website is updated, these queries may not work and would need to be altered.
    
    #setName = "Ixalan"
    rmList = ["R","M"]
    numCards = 0
    cardNames = [[],[]]
    CMCs = []
    cardTypes = []
    cardSubtypes = []
    powers = []
    toughnesses = []
    loyalties = []
    cardText = []
    for i in rmList:
        URL = "http://gatherer.wizards.com/Pages/Search/Default.aspx?action=advanced&set=+[%22"+setName +"%22]&rarity=+["+ i + "]"
        r = requests.get(URL)
        HTML = r.text
        numCards = int(HTML.split('searchTermDisplay')[3].split("(")[1].split(')')[0])
        #print(numCards)
        
        for x in range(numCards):
            go = True
            if i == "R":
                if HTML.split('img title="'+setName+" (")[x+1].split(')')[0] != "Rare":
                    go = False
            elif i == "M":
                if HTML.split('img title="'+setName+" (")[x+1].split(')')[0] != "Mythic Rare":
                    go = False
            if go == True:
                tempName = HTML.split('</a></span> <span class="manaCost">')[x].split('>')[-1]
                tempCMC = HTML.split('<span class="convertedManaCost">')[x+1].split('</span>)\r\n ')[0]
                tempTypes, tempSubtypes = tld(HTML.split('<span class="typeLine">')[x+1].split('\r\n')[1].split(" "))
                try:
                    tempText = importantRulesTextExtraction(HTML.split('<div class="rulesText">\r\n')[x+1].split('</div>')[0])
                except:
                    print("ERROR SOURCE!!!")
                    print(HTML.split('<div class="rulesText">\r\n')[x+1].split('</div>')[0])
                if HTML.split('<span class="typeLine">')[x+1].split('\r\n')[2].split('</span>')[0].split(" ")[-1] != "":
                    tempPT = HTML.split('<span class="typeLine">')[x+1].split('\r\n')[2].split('</span>')[0].split(" ")[-1].split("(")[1].split(")")[0].split("/")
                    #print(tempPT)
                    tempPow = tempPT[0]
                    try:
                        tempTough = tempPT[1]
                        powers.append(tempPow)
                        toughnesses.append(tempTough)
                        loyalties.append(-1)
                    except:
                        tempLoyalty = tempPow
                        powers.append(-1)
                        toughnesses.append(-1)
                        loyalties.append(tempLoyalty)
                else:
                    powers.append(-1)      #N/A
                    toughnesses.append(-1) #N/A
                    loyalties.append(-1)   #N/A
                #print(tempName)
                cardNames[rmList.index(i)].append(tempName)
                CMCs.append(tempCMC)
                cardTypes.append(tempTypes)
                cardSubtypes.append(tempSubtypes)
                cardText.append(tempText)
        #print(cardNames)
    #rareNames,mythicNames --> out -- these are for searching on mtggoldfish.com and will be altered.
    rareNames = cardNames[0]
    mythicNames = cardNames[1]
    
    
    if setName == "Ixalan":
        removeList = ["Adanto, the First Fort", "Azcanta, the Sunken Ruin","Conqueror's Foothold",
                     "Itlimoc, Cradle of the Sun","Lost Vale","Primal Wellspring",
                     "Spires of Orazca","Spitfire Bastion","Temple of Aclazotz",
                     "Treasure Cove"]
        for i in removeList:
            rareNames.remove(i)
    
    #these would've been the unaltered real names of the cards.
    #rareList = cardNames[0][:]
    #mythicList = cardNames[1][:]
    
    #removing punctuation
    rareNames = removePunctuationFromList(rareNames)
    mythicNames = removePunctuationFromList(mythicNames)
    allNames = rareNames+mythicNames
    
    #while the + are still not in the name, let's replace instance's of the full exact card name within the rules
    #of the card as 'selfReference'
    for i in range(len(allNames)):
        if allNames[i] in cardText[i]:
            cardText[i] = cardText[i].replace(allNames[i]," selfReference ")
            
    #replacing " " with "+"
    rareNames = spacesToPlus(rareNames)
    mythicNames = spacesToPlus(mythicNames)
    
    #cleaning up CMCs list and other lists by converting the strings to the integers they should be.
    CMCs = listToInt(CMCs)
    powers = listToInt(powers)
    toughnesses = listToInt(toughnesses)
    loyalties = listToInt(loyalties)
    
    #the first two here are altered name lists.  The last two are the unaltered, real names of the cards.
    return rareNames,mythicNames,CMCs,cardTypes,cardSubtypes,powers,toughnesses,loyalties,cardText#, rareList, mythicList

In [49]:
# sn = "Theros"
# rareNames,mythicNames,CMCs,cardTypes,cardSubtypes,powers,toughnesses,loyalties,cardText = getRareAndMythicInfo(sn)
# print(rareNames)
# print(mythicNames)
# print(CMCs)
# print(cardTypes)
# print(cardSubtypes)
# print(powers)
# print(toughnesses)
# print(loyalties)
# print(cardText)

In [50]:
# cvec = CountVectorizer(binary=True)
# cvec.fit(stuff)

# df  = pd.DataFrame(cvec.transform(stuff).todense(),columns=cvec.get_feature_names())
# df.head(5)

In [51]:
def getCurrentCardValues(listOfCardNames,setName):
    '''generalized function to grab all the current card prices for a list of card names.'''
    cardPrices = []
    for i in range(len(listOfCardNames)):
        URL = 'https://www.mtggoldfish.com/price/'+setName+'/'+listOfCardNames[i]
        r = requests.get(URL)
        HTML = r.text
        cardPrices.append(HTML.split("<div class='price-box-type'>PAPER</div>")[1].split('price-box-price')[1].split('>')[1].split('<')[0])
        #print(HTML.split("<div class='price-box-type'>PAPER</div>")[1].split('price-box-price')[1].split('>')[1].split('<')[0])
    return cardPrices

In [65]:
def setScraper(setName,prices=False):
    '''The main function! Enter a set name!
    
    This function will use all the above functions to pull an entire set of mtg card names
    from gatherer.wizards.com, and then pull all the current prices from mtggoldfish.com, and output all of it
    neatly into a pandas DataFrame for you!
    
    If you dont want prices, keep it set to False, if you do, set it to True!'''
    rareNames,mythicNames,CMCs,cardTypes,cardSubtypes,powers,toughnesses,loyalties,cardText = getRareAndMythicInfo(setName)
    #print(rareNames)
    #print(mythicNames)
    alteredSetName = setName.replace(" ","+")
    #print(alteredSetName)
    if prices == True:
        rarePrices = getCurrentCardValues(rareNames,alteredSetName)
        mythicPrices = getCurrentCardValues(mythicNames,alteredSetName)

        for i in range(len(rarePrices)):
            rarePrices[i] = float(rarePrices[i])

        for i in range(len(mythicPrices)):
            mythicPrices[i] = float(mythicPrices[i])
    
    rareLabeler = ["Rare"]*len(rareNames)
    mythicLabeler = ["Mythic"]*len(mythicNames)
    
    df = pd.DataFrame()
    
    #removing pluses from names
    rareNames = plusToSpaces(rareNames)
    mythicNames = plusToSpaces(mythicNames)
    
    #making all columns of DataFrame
    df["cardName"] = rareNames+mythicNames
    df["rarity"] = rareLabeler+mythicLabeler
    if prices == True:
        df["currentPrice"] = rarePrices+mythicPrices
        df["timeOfDataScrape"] = [datetime.datetime.now()]*len(powers)
    df["convertedManaCost"] = CMCs
    df["powers"] = powers
    df["toughnesses"] = toughnesses
    df["loyalties"] = loyalties
    df["cardText"] = cardText
    
    knownTypes = ["Creature","Legendary","Artifact","Instant","Sorcery","Enchantment","Planeswalker",
                 "Land","Equipment","Tribal","Conspiracy","World","Ongoing","Phenomenon","Plane",
                 "Scheme","Snow","Vanguard","Basic"]
    
    for i in range(len(knownTypes)):
        df[knownTypes[i]] = [0]*len(powers)
        df["Other"] = [-1]*len(powers)
    
    for i in range(len(cardTypes)):
        try:
            for j in range(len(cardTypes[i])):
                df[cardTypes[i][j]][i] += 1
        except:
            df["Other"][i] = cardTypes[i]
    
    return df

In [66]:
def saveFile(df,setName):
    '''just use the name of the set and it will save it in the proper place with the proper extension.'''
    df.to_csv("./csvs/"+setName+".csv")

In [73]:
def importFile(setName):
    '''use name of the set you want to import data for.  
    This function returns a pandas DataFrame.'''
    df = pd.read_csv("./csvs/"+setName+".csv")
    try:
        df = df.drop("Unnamed: 0",axis=1)
    except:
        print("There was not a column named Unnamed: 0 to drop")
    return df

In [74]:
Theros = setScraper("Theros")
saveFile(Theros,"Theros")

/home/tpeach/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [75]:
Theros.head(20)

,cardName,rarity,convertedManaCost,powers,toughnesses,loyalties,cardText,Creature,Other,Legendary,...,Tribal,Conspiracy,World,Ongoing,Phenomenon,Plane,Scheme,Snow,Vanguard,Basic
0,Abhorrent Overlord,Rare,7,6,6,-1,Flying When ...,1,-1,0,...,0,0,0,0,0,0,0,0,0,0
1,Agent of the Fates,Rare,3,3,2,-1,Deathtouch H...,1,-1,0,...,0,0,0,0,0,0,0,0,0,0
2,Akroan Horse,Rare,4,0,4,-1,Defender Whe...,1,-1,0,...,0,0,0,0,0,0,0,0,0,0
3,Anax and Cymede,Rare,3,3,2,-1,"First strike,...",1,-1,1,...,0,0,0,0,0,0,0,0,0,0
4,Anger of the Gods,Rare,3,-1,-1,-1,selfReferenc...,0,-1,0,...,0,0,0,0,0,0,0,0,0,0
5,Anthousa Setessan Hero,Rare,5,4,5,-1,Heroic — When...,1,-1,1,...,0,0,0,0,0,0,0,0,0,0
6,Arbor Colossus,Rare,5,6,6,-1,Reach thre...,1,-1,0,...,0,0,0,0,0,0,0,0,0,0
7,Artisan of Forms,Rare,2,1,1,-1,Heroic — When...,1,-1,0,...,0,0,0,0,0,0,0,0,0,0
8,Bident of Thassa,Rare,4,-1,-1,-1,Whenever a cr...,0,-1,1,...,0,0,0,0,0,0,0,0,0,0
9,Boon Satyr,Rare,3,4,2,-1,Flash Bestow...,1,-1,0,...,0,0,0,0,0,0,0,0,0,0


In [76]:
Theros["cardText"][0]

'                                 Flying  When  selfReference  enters the battlefield, create a number of  one  slashDivider  one  black Harpy creature tokens with flying equal to your devotion to black.  Each  blackMana  in the mana costs of permanents you control counts toward your devotion to black.   At the beginning of your upkeep, sacrifice a creature. '

In [77]:
testDfPlz = importFile("Theros")

In [78]:
testDfPlz.head()

,cardName,rarity,convertedManaCost,powers,toughnesses,loyalties,cardText,Creature,Other,Legendary,...,Tribal,Conspiracy,World,Ongoing,Phenomenon,Plane,Scheme,Snow,Vanguard,Basic
0,Abhorrent Overlord,Rare,7,6,6,-1,Flying When ...,1,-1,0,...,0,0,0,0,0,0,0,0,0,0
1,Agent of the Fates,Rare,3,3,2,-1,Deathtouch H...,1,-1,0,...,0,0,0,0,0,0,0,0,0,0
2,Akroan Horse,Rare,4,0,4,-1,Defender Whe...,1,-1,0,...,0,0,0,0,0,0,0,0,0,0
3,Anax and Cymede,Rare,3,3,2,-1,"First strike,...",1,-1,1,...,0,0,0,0,0,0,0,0,0,0
4,Anger of the Gods,Rare,3,-1,-1,-1,selfReferenc...,0,-1,0,...,0,0,0,0,0,0,0,0,0,0
